# Building your deep neural network:
step by step

In this Notebook, you implement all the function required to build a deep neural network

# Notation

1.Superscript[l] denotes a quantity associated with the lth layer.
 
 Example: a[l] is the Lth layer activation.W[l] and b[l] are the Lth layer parameters.


2.Superscript(i)  denotes a quantity associated with the ith example.

Example: x[i] is the ith training example


3.Lowerscript i denote the ith entry of a vector
  

# packages

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0,4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'grey'

np.random.seed(1)

In [2]:
%run dnn_utils.ipynb

# Initialization

we will write two helper function that will initialize the parameter for model.

The first function will be used to initialize parameter for 2 layer model. 

The second one will generlize this initialization process to L layer

In [3]:
#  Graded function : initialize_parameter

def initialize_parameter(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of input layer
    n_h -- size of hidden layer
    n_y -- size of output layer

    Return:
    parameters -- python dictionary containing parameters:
                   W1 -- weight matrix of shape (n_h, n_x)
                   b1 -- bias of matrix of shape (n_h, 1)
                   W2 -- weight matrix of shape (n_h, n_y)
                   b2 -- bias matrix of shape (n_y, 1)
                   """
    np.random.seed(1)
    ## start code here ##
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    parameters = {'W1':W1,'b1':b1,'W2':W2,'b2':b2}
    return parameters

In [4]:
parameters = initialize_parameter(3,2,1)

In [5]:
parameters

{'W1': array([[ 0.01624345, -0.00611756, -0.00528172],
        [-0.01072969,  0.00865408, -0.02301539]]),
 'b1': array([[0.],
        [0.]]),
 'W2': array([[ 0.01744812, -0.00761207]]),
 'b2': array([[0.]])}

# L-layer neural network

Exercise - Implement initialization for an L layer Neural Network

In [6]:
def initialize_parameter_deep(layer_dims):
    """
    Argument:
    layer_dims -- python array (list) containing the dimensions of each layer in our network

    Return:
    parameters -- python dictionary containing your parameters "W1","b1",...."WL","bL":
    """
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((layer_dims[l],1))
        ## end code here##
        assert(parameters['W'+str(l)].shape == (layer_dims[l],layer_dims[l-1]))
        assert(parameters['b'+str(l)].shape == (layer_dims[l],1))
    return parameters

In [7]:
parameters = initialize_parameter_deep([5,4,3,1,5,2,1])

In [20]:
len(parameters)//2

6

# Forward propagation module

Exercise - Build linear part of forward propagation

In [9]:
def linear_forward(A,W,b):
    """
    Implement the linear part of a layer's forward propagation

    Argument:
    A -- Activation from previous layer (or input data):(size of previous layer, number of example)
    W -- weight matrix: (size of current layer,size of A)
    b -- bias vector : (size of current layer , 1)
    Return:
    Z -- the input of the activation function
    cache -- a python tuple containing "A","W" and b
    """
    Z = np.dot(W,A) + b
    assert(Z.shape == (W.shape[0],A.shape[1]))
    cache = (A,W,b)
    return Z,cache

In [11]:
A = np.random.randn(4,10)
W = np.random.randn(3,4)
b = np.random.randn(3,1)

In [12]:
Z,cache = linear_forward(A,W,b)


In [13]:
Z

array([[-0.91156237, -0.37064181, -1.17954225, -2.11397941, -2.31412053,
        -2.03525026, -2.49121645, -0.12504553,  0.10917142, -1.31357662],
       [ 0.61560635,  0.01951302,  0.20523496,  1.30121351, -0.34806663,
        -0.33431456,  0.10884262,  0.12010891,  1.34055284, -0.02969873],
       [ 2.05480795,  1.12866528,  1.67721578, -0.35957799,  1.22947042,
         2.74345612,  3.45426792, -0.38733344, -0.0685148 , -0.40845835]])

In [14]:
cache

(array([[-1.1425182 , -0.34934272, -0.20889423,  0.58662319,  0.83898341,
          0.93110208,  0.28558733,  0.88514116, -0.75439794,  1.25286816],
        [ 0.51292982, -0.29809284,  0.48851815, -0.07557171,  1.13162939,
          1.51981682,  2.18557541, -1.39649634, -1.44411381, -0.50446586],
        [ 0.16003707,  0.87616892,  0.31563495, -2.02220122, -0.30620401,
          0.82797464,  0.23009474,  0.76201118, -0.22232814, -0.20075807],
        [ 0.18656139,  0.41005165,  0.19829972,  0.11900865, -0.67066229,
          0.37756379,  0.12182127,  1.12948391,  1.19891788,  0.18515642]]),
 array([[-0.37528495, -0.63873041,  0.42349435,  0.07734007],
        [-0.34385368,  0.04359686, -0.62000084,  0.69803203],
        [-0.44712856,  1.2245077 ,  0.40349164,  0.59357852]]),
 array([[-1.09491185],
        [ 0.16938243],
        [ 0.74055645]]))

# Linear-Activation forward

Exercise - Implement the forward propagation of linear Activation layer.

In [15]:
# Graded function : linear_activation_forward
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the Linear ->Activation layer

    Arguments:
    A_prev -- activation from previous layer or input data:(size of previous layer,number of example)
    W --- weight matrix
    b -- bias matrix
    activation -- the activation used in this layer

    Return:
    A -- the output of the activation function, also called post-activation value
    cache -- a python tuple containing 'linear_cache' and 'activation_cache'
    """
    if activation == 'sigmoid':
        Z,linear_cache = linear_forward(A_prev, W, b)
        A,activation_cache = sigmoid(Z)
        ## End code here'##
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ## end code here ##
    assert (A.shape ==(W.shape[0],A_prev.shape[1]))
    cache = (linear_cache,activation_cache)

    return A, cache
        

In [16]:
A1,cache = linear_activation_forward(A,W,b,'sigmoid')

In [17]:
A1

array([[0.28668023, 0.40838595, 0.23513451, 0.1077455 , 0.08996024,
        0.11555127, 0.07647624, 0.46877929, 0.52726578, 0.21188896],
       [0.64921862, 0.5048781 , 0.5511294 , 0.78603914, 0.41385134,
        0.4171912 , 0.52718382, 0.52999118, 0.79258084, 0.49257586],
       [0.88643253, 0.7555925 , 0.8425355 , 0.41106173, 0.77372587,
        0.93954271, 0.96935816, 0.40435939, 0.482878  , 0.39928184]])

In [18]:
cache

((array([[-1.1425182 , -0.34934272, -0.20889423,  0.58662319,  0.83898341,
           0.93110208,  0.28558733,  0.88514116, -0.75439794,  1.25286816],
         [ 0.51292982, -0.29809284,  0.48851815, -0.07557171,  1.13162939,
           1.51981682,  2.18557541, -1.39649634, -1.44411381, -0.50446586],
         [ 0.16003707,  0.87616892,  0.31563495, -2.02220122, -0.30620401,
           0.82797464,  0.23009474,  0.76201118, -0.22232814, -0.20075807],
         [ 0.18656139,  0.41005165,  0.19829972,  0.11900865, -0.67066229,
           0.37756379,  0.12182127,  1.12948391,  1.19891788,  0.18515642]]),
  array([[-0.37528495, -0.63873041,  0.42349435,  0.07734007],
         [-0.34385368,  0.04359686, -0.62000084,  0.69803203],
         [-0.44712856,  1.2245077 ,  0.40349164,  0.59357852]]),
  array([[-1.09491185],
         [ 0.16938243],
         [ 0.74055645]])),
 array([[-0.91156237, -0.37064181, -1.17954225, -2.11397941, -2.31412053,
         -2.03525026, -2.49121645, -0.12504553,  0.109

# L- layer Model

Exercise -- Implement forward propagation for L-layer model

In [27]:
def L_model_forward(X,parameters):
    """
    Implement forward propagation for the [Linear ->Relu]*(L-1)->Linear ->Sigmoid computation

    Argument:
    X -- data, numpy array of shape (input size, number of example)
    parameters -- output of initialize_parameters_deep()

    Return:
    A[L] -- last post-activation value
    cache -- list of cache containing:
              every cache of linear_activation_forward()
    """
    caches = []
    A = X
    L = len(parameters)//2
    # implement [Linear -> relu]*(L-1)
    for l in range(1,L):
        A_prev = A
        A,cache = linear_activation_forward(A_prev,parameters['W'+str(l)],parameters['b'+str(l)],'relu')
        caches.append(cache)
        ## end code here

    # Implement Linear -> sigmoid. ##
    AL, cache = linear_activation_forward(A,parameters['W'+str(L)],parameters['b'+str(L)],'sigmoid')
    caches.append(cache)
    assert(AL.shape == (1,X.shape[1]))
    return AL,caches
     
        

In [22]:
def L_model_forward_test_case_2hidden():
    np.random.seed(6)
    X = np.random.randn(5,4)
    W1 = np.random.randn(4,5)
    b1 = np.random.randn(4,1)
    W2 = np.random.randn(3,4)
    b2 = np.random.randn(3,1)
    W3 = np.random.randn(1,3)
    b3 = np.random.randn(1,1)
  
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3}
    
    return X, parameters

In [23]:
X, parameters = L_model_forward_test_case_2hidden()

In [28]:
AL,caches = L_model_forward(X,parameters)
print('AL =' + str(AL))
print('Length of caches list =' + str(len(caches)),'caches =',caches)

AL =[[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list =3 caches = [((array([[-0.31178367,  0.72900392,  0.21782079, -0.8990918 ],
       [-2.48678065,  0.91325152,  1.12706373, -1.51409323],
       [ 1.63929108, -0.4298936 ,  2.63128056,  0.60182225],
       [-0.33588161,  1.23773784,  0.11112817,  0.12915125],
       [ 0.07612761, -0.15512816,  0.63422534,  0.810655  ]]), array([[ 0.35480861,  1.81259031, -1.3564758 , -0.46363197,  0.82465384],
       [-1.17643148,  1.56448966,  0.71270509, -0.1810066 ,  0.53419953],
       [-0.58661296, -1.48185327,  0.85724762,  0.94309899,  0.11444143],
       [-0.02195668, -2.12714455, -0.83440747, -0.46550831,  0.23371059]]), array([[ 1.38503523],
       [-0.51962709],
       [-0.78015214],
       [ 0.95560959]])), array([[-5.23825714,  3.18040136,  0.4074501 , -1.88612721],
       [-2.77358234, -0.56177316,  3.18141623, -0.99209432],
       [ 4.18500916, -1.78006909, -0.14502619,  2.72141638],
       [ 5.05850802, -1.25674082

Great! Now we have a full forward propagation that take the input X and output a row vector A[l] containing prediction. It also records all intermediate value in cache.

Let's compute cost function

# Cost function

Exercise -- Compute the cross-entropy cost J

In [29]:
# graded function : compute_cost
def compute_cost(AL,Y):
    """
    Implement the cost function defined.

    Argument:
    AL -- probability vector corrosponding to label prediction
    Y -- true label vector

    Return:
    cost -- cross entropy cost
    """
    m = Y.shape[1]
    # compute Loss from AL and Y
    cost = (-1/m) * (np.dot(Y,np.log(AL).T) + np.dot((1-Y),np.log(1-AL).T))
    ## End code here##
    cost = np.squeeze(cost)
    assert(cost.shape ==())
    return cost

In [30]:
def compute_cost_test_case():
    Y = np.asarray([[1, 1, 0]])
    aL = np.array([[.8,.9,0.4]])
    
    return Y, aL

In [31]:
Y, AL = compute_cost_test_case()

In [32]:
print('cost = ',compute_cost(AL,Y))

cost =  0.2797765635793422


# Backward Propagation

Just like forward propagation, we will implement helper function for backpropagation

In [33]:
# graded function: linear_backward
def linear_backward(dZ,cache):
    # Here cache is linear_cache" containing (A_prev,W,b) coming from the forward propagation in current layer
    """ 
    Implement the linear portion of backward propagation for a single layer(l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output of current layer 1
    cache -- tuple of value (A_prev, W, b) coming from the forward propagation in current layer

    Return:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1),same shape as A_prev
    dW -- Gradient of the cost with respect to W
    db -- gradient of the cost with respect to b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]
    ## start code here ##
    dW = (1/m) * np.dot(dZ,A_prev.T)
    db = (1/m) * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    ## end code here ##
    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(db.shape == b.shape)
    return dA_prev, dW, db

In [34]:
def linear_backward_test_case():
    """
    z, linear_cache = (np.array([[-0.8019545 ,  3.85763489]]), (np.array([[-1.02387576,  1.12397796],
       [-1.62328545,  0.64667545],
       [-1.74314104, -0.59664964]]), np.array([[ 0.74505627,  1.97611078, -1.24412333]]), np.array([[1]]))
    """
    np.random.seed(1)
    dZ = np.random.randn(3,4)
    A = np.random.randn(5,4)
    W = np.random.randn(3,5)
    b = np.random.randn(3,1)
    linear_cache = (A, W, b)
    return dZ, linear_cache

In [35]:
# set up some test inputs

dZ, linear_cache = linear_backward_test_case()


In [36]:
dA_prev, dW, db = linear_backward(dZ,linear_cache)
print('dA_prev =',dA_prev)
print('dW =',dW)
print('db =',db)

dA_prev = [[-1.15171336  0.06718465 -0.3204696   2.09812712]
 [ 0.60345879 -3.72508701  5.81700741 -3.84326836]
 [-0.4319552  -1.30987417  1.72354705  0.05070578]
 [-0.38981415  0.60811244 -1.25938424  1.47191593]
 [-2.52214926  2.67882552 -0.67947465  1.48119548]]
dW = [[ 0.07313866 -0.0976715  -0.87585828  0.73763362  0.00785716]
 [ 0.85508818  0.37530413 -0.59912655  0.71278189 -0.58931808]
 [ 0.97913304 -0.24376494 -0.08839671  0.55151192 -0.10290907]]
db = [[-0.14713786]
 [-0.11313155]
 [-0.13209101]]


# Linear-Activation backward 

Next, you will create a function that merges the two helper functions:

linear_backward and the backward step for the activation

linear_activation_backward .

In [42]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the Linear -> Activation layer.

    Argument:
    dA -- post-activation gradient for current layer l
    cache -- tuple of values (linear_cache,activation_cache)
    activation -- the activation to be used in this layer

    Return:
    dA_prev , db, dW
    """
    linear_cache, activation_cache = cache
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA,activation_cache)
        ## end code here ##
    dA_prev, dW, db = linear_backward(dZ,linear_cache)
    return dA_prev, dW, db

In [38]:
def linear_activation_backward_test_case():
    """
    aL, linear_activation_cache = (np.array([[ 3.1980455 ,  7.85763489]]), ((np.array([[-1.02387576,  1.12397796], [-1.62328545,  0.64667545], [-1.74314104, -0.59664964]]), np.array([[ 0.74505627,  1.97611078, -1.24412333]]), 5), np.array([[ 3.1980455 ,  7.85763489]])))
    """
    np.random.seed(2)
    dA = np.random.randn(1,2)
    A = np.random.randn(3,2)
    W = np.random.randn(1,3)
    b = np.random.randn(1,1)
    Z = np.random.randn(1,2)
    linear_cache = (A, W, b)
    activation_cache = Z
    linear_activation_cache = (linear_cache, activation_cache)
    
    return dA, linear_activation_cache

In [39]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

In [43]:
dA_prev, dW, db = linear_activation_backward(dAL,linear_activation_cache,activation = 'sigmoid')
print('sigmoid:')
print('dA_prev = ',dA_prev)
print('dW =',dW)
print('db =',db)

sigmoid:
dA_prev =  [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]


# L-Model Backward

In [48]:
# Graded function: l_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [Linear ->Relu]*(L-1) -> sigmoid group

    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true 'label' vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                        every caches of linear_activation_forward() with 'relu'(it's caches[l], for l in range(L-1))

    Return:
    grads -- A dictionary with the gradients
              grads['dA'+str(l)]
              grads['dW'+str(l)]
              grads['db'+str(l)]
     """
    grads = {}
    L = len(caches) 
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    # 'initializing the backpropagation'
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))
    # End code here
    # Lth layer (sigmoid ->Linear)gradient. 
    current_cache = caches[L-1] # Last Layer
    grads['dA' + str(L-1)],grads['dW'+str(L)],grads['db'+str(L)] = linear_activation_backward(dAL, current_cache,'sigmoid')
    ## 'end code here'

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads['dA',str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
    return grads    
        
        


In [45]:
def L_model_backward_test_case():
    """
    X = np.random.rand(3,2)
    Y = np.array([[1, 1]])
    parameters = {'W1': np.array([[ 1.78862847,  0.43650985,  0.09649747]]), 'b1': np.array([[ 0.]])}

    aL, caches = (np.array([[ 0.60298372,  0.87182628]]), [((np.array([[ 0.20445225,  0.87811744],
           [ 0.02738759,  0.67046751],
           [ 0.4173048 ,  0.55868983]]),
    np.array([[ 1.78862847,  0.43650985,  0.09649747]]),
    np.array([[ 0.]])),
   np.array([[ 0.41791293,  1.91720367]]))])
   """
    np.random.seed(3)
    AL = np.random.randn(1, 2)
    Y = np.array([[1, 0]])

    A1 = np.random.randn(4,2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    Z1 = np.random.randn(3,2)
    linear_cache_activation_1 = ((A1, W1, b1), Z1)

    A2 = np.random.randn(3,2)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)
    Z2 = np.random.randn(1,2)
    linear_cache_activation_2 = ((A2, W2, b2), Z2)

    caches = (linear_cache_activation_1, linear_cache_activation_2)

    return AL, Y, caches


In [50]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL,Y_assess, caches)
print(grads)

{'dA1': array([[ 0.12913162, -0.44014127],
       [-0.14175655,  0.48317296],
       [ 0.01663708, -0.05670698]]), 'dW2': array([[-0.39202432, -0.13325855, -0.04601089]]), 'db2': array([[0.15187861]]), ('dA', '0'): array([[ 0.        ,  0.52257901],
       [ 0.        , -0.3269206 ],
       [ 0.        , -0.32070404],
       [ 0.        , -0.74079187]]), 'dW1': array([[0.41010002, 0.07807203, 0.13798444, 0.10502167],
       [0.        , 0.        , 0.        , 0.        ],
       [0.05283652, 0.01005865, 0.01777766, 0.0135308 ]]), 'db1': array([[-0.22007063],
       [ 0.        ],
       [-0.02835349]])}


# Update parameters

Exercise -- Implement update_parameters() to update your parameters using gradient descent

In [52]:
# Graded function: update_parameters
def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent

    Argument:
    parameters -- python dictionary containing parameters
    grads -- Output of L-model

    Return:
    parameters -- python dictionary containing updated parameters
    """
    L = len(parameters)//2
    for l in range(L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate * grads['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate * grads['db'+str(l+1)]
        # 'end code here' #
    return parameters 
        
        

In [53]:
def update_parameters_test_case():
    """
    parameters = {'W1': np.array([[ 1.78862847,  0.43650985,  0.09649747],
        [-1.8634927 , -0.2773882 , -0.35475898],
        [-0.08274148, -0.62700068, -0.04381817],
        [-0.47721803, -1.31386475,  0.88462238]]),
 'W2': np.array([[ 0.88131804,  1.70957306,  0.05003364, -0.40467741],
        [-0.54535995, -1.54647732,  0.98236743, -1.10106763],
        [-1.18504653, -0.2056499 ,  1.48614836,  0.23671627]]),
 'W3': np.array([[-1.02378514, -0.7129932 ,  0.62524497],
        [-0.16051336, -0.76883635, -0.23003072]]),
 'b1': np.array([[ 0.],
        [ 0.],
        [ 0.],
        [ 0.]]),
 'b2': np.array([[ 0.],
        [ 0.],
        [ 0.]]),
 'b3': np.array([[ 0.],
        [ 0.]])}
    grads = {'dW1': np.array([[ 0.63070583,  0.66482653,  0.18308507],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ]]),
 'dW2': np.array([[ 1.62934255,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ]]),
 'dW3': np.array([[-1.40260776,  0.        ,  0.        ]]),
 'da1': np.array([[ 0.70760786,  0.65063504],
        [ 0.17268975,  0.15878569],
        [ 0.03817582,  0.03510211]]),
 'da2': np.array([[ 0.39561478,  0.36376198],
        [ 0.7674101 ,  0.70562233],
        [ 0.0224596 ,  0.02065127],
        [-0.18165561, -0.16702967]]),
 'da3': np.array([[ 0.44888991,  0.41274769],
        [ 0.31261975,  0.28744927],
        [-0.27414557, -0.25207283]]),
 'db1': 0.75937676204411464,
 'db2': 0.86163759922811056,
 'db3': -0.84161956022334572}
    """
    np.random.seed(2)
    W1 = np.random.randn(3,4)
    b1 = np.random.randn(3,1)
    W2 = np.random.randn(1,3)
    b2 = np.random.randn(1,1)
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    np.random.seed(3)
    dW1 = np.random.randn(3,4)
    db1 = np.random.randn(3,1)
    dW2 = np.random.randn(1,3)
    db2 = np.random.randn(1,1)
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return parameters, grads


In [54]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads,0.1)
print('W1 =',parameters['W1'])
print('b1 =',parameters['b1'])
print('W2 =',parameters['W2'])
print('b2 =',parameters['b2'])

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


# Conclusion

Congrats on implementing all the function required for building a deep neural network!

